In [40]:
import clickhouse_connect
import pandas as pd
import numpy as np

In [53]:
# Read the data
#data_file = "TimeSeries/cu/cu.data.1.Allitems"
data_file = "TimeSeries/cu/cu.data.0.Current"
df = pd.read_csv(data_file, sep='\t')
# remove spaces from names
df.columns = df.columns.str.strip()
# remove spaces from series_id
df['series_id'] = df['series_id'].str.strip()

In [49]:
# Generate SQL to create the table
pandas_to_clickhouse_dtypes = {
    'object': 'String',
    'int64': 'Int64',
    'float64': 'Float64',
    'bool': 'UInt8',
    'datetime64[ns]': 'DateTime',
    'timedelta64[ns]': 'Interval',  # Stores duration like days, hours, etc.
    'category': 'String',  # Category data can be complex, may need conversion
    'string': 'String',
    'Int8Dtype': 'Int8',
    'Int16Dtype': 'Int16',
    'Int32Dtype': 'Int32',
    'UInt8Dtype': 'UInt8',
    'UInt16Dtype': 'UInt16',
    'UInt32Dtype': 'UInt32',
}
table_columns = []
for name, pd_type in df.dtypes.items():
    click_type = pandas_to_clickhouse_dtypes[pd_type.name]
    if df[name].isnull().any():
        click_type = f" Nullable({click_type})"
    column_definition = f"{name} {click_type}"
    table_columns.append(column_definition)

table_name = 'bls.tbl_cu_1'
sql = f"CREATE TABLE {table_name}({','.join(table_column_types)}) ENGINE = MergeTree() PRIMARY KEY (series_id, year, period)"
print(sql)


CREATE TABLE bls.tbl_cu_1(series_id String,year Int64,period String,value Float64,footnote_codes  Nullable(Float64)) ENGINE = MergeTree() PRIMARY KEY (series_id, year, period)


In [47]:
# create new table
client = clickhouse_connect.get_client(host='localhost', port=8123, username='default', password='', database='bls')
client.command(f"DROP TABLE IF EXISTS {table_name}")
client.command(sql)

In [48]:
# load data with np.nan replace with None
client.insert_df(table_name, df.replace({np.nan: None}))